# Federated Cluster Model with Flex  for Anomaly Detection

In [1]:
from flexanomalies.models import ClusterAnomaly
from flexanomalies.utils.load_data import load_and_split_dot_mat, federate_data
from flexanomalies.pool.aggregators_cl import aggregate_cl
from flexanomalies.pool.decorators_cluster import (
    build_server_model_cl,
    copy_model_to_clients_cl,
    train_cl,
    set_aggregated_weights_cl,
    get_clients_weights_cl,
)
from flexanomalies.utils.save_results import save_experiments_results
from flex.pool import FlexPool

2023-12-07 12:29:11.242846: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-07 12:29:11.353036: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-07 12:29:11.355986: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-07 12:29:11.356002: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
model_params = {"n_clusters": 4, "contamination": 0.1}

In [3]:
X_train, X_test, y_train, y_test = load_and_split_dot_mat(
    "flexanomalies/datasets/data/shuttle.mat", 0.3
)

In [4]:
model = ClusterAnomaly(**model_params)

In [5]:
flex_dataset = federate_data(5, X_train, y_train)

In [6]:
pool = FlexPool.client_server_pool(
    fed_dataset=flex_dataset,
    server_id="cluster_server",
    init_func=build_server_model_cl,
    model=model,
)

In [9]:
for i in range(3):
    print(f"\nRunning round: {i}\n")
    pool.servers.map(copy_model_to_clients_cl, pool.clients)
    pool.clients.map(train_cl)
    pool.aggregators.map(get_clients_weights_cl, pool.clients)
    pool.aggregators.map(aggregate_cl, model=model)
    pool.aggregators.map(set_aggregated_weights_cl, pool.servers)
output_model = pool.servers._models["cluster_server"]["model"]
output_model.evaluate(X_test, y_test)
print()


Running round: 0

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 1

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 2

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.
Acc: 94.365% 

Precision: 0.585 

F1score: 0.675 

Recall: 0.797 

AUC_ROC: 0.876 




In [10]:
output_model.result_metrics_

{'Accuracy': '94.365%',
 'Precision': '0.585',
 'F1': '0.675',
 'Recall': '0.797',
 'AUC_ROC': '0.876'}

In [11]:
save_experiments_results(
    "cluster",
    output_model,
    "test_cluster_notebook",
    model_params,
    "shuttle.mat",
    5,
    3,
    0.3,
)